In [ ]:
!pip install -U ipykernel

     |████████████████████████████████| 122kB 19.0MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 7.5MB 15.3MB/s 
     |████████████████████████████████| 112kB 56.8MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 4.6MB 48.6MB/s 
     |████████████████████████████████| 163kB 43.7MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 


In [ ]:
!pip install pyngrok==4.1.1

  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=511199107d14aa3861e2c65ae4cafb940121671311909c005a85d3dc5aad123c
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile /content/drive/MyDrive/seq2py/score_glove_augmentation.py


import torch
from torchtext import data 
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import unicodedata
import codecs
import csv
import time
import random
import re
import os
import pickle
from io import open
import itertools
import math
import pandas as pd
from torchtext.legacy.data import Field, BucketIterator, LabelField, TabularDataset
import streamlit as st

BASE_DIR='/content/drive/MyDrive/seq2py'
#https://stackoverflow.com/questions/62922640/calling-a-function-in-a-different-python-file-using-google-colab
# %cd $BASE_DIR/utils
import utils
from utils import preprocess, helper,transformer
from utils.transformer import Encoder,Decoder,Seq2Seq
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

st.title("Generate python code from english text")

with open(os.path.join(BASE_DIR ,"model/src_stio.pkl"),"rb") as f:
  src_stoi = pickle.load(f)
with open(os.path.join(BASE_DIR ,"model/src_itos.pkl"),"rb") as f:
  src_itos = pickle.load(f)


with open(os.path.join(BASE_DIR ,"model/trg_stio.pkl"),"rb") as f:
  trg_stoi = pickle.load(f)
with open(os.path.join(BASE_DIR ,"model/trg_itos.pkl"),"rb") as f:
  trg_itos = pickle.load(f)

INPUT_DIM = len(src_stoi)
OUTPUT_DIM = len(trg_stoi)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 4
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)


dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT,
              device)

SRC_PAD_IDX = src_stoi['<pad>']
TRG_PAD_IDX = trg_stoi['<pad>']

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)



@st.cache(suppress_st_warning=True,ttl=1000)
def load_model(model_path):
    # Load model
    trained_model = os.path.join(model_path,"model/tut6-model_20210311_run4_glove_moredata.pt")
    model.load_state_dict(torch.load(trained_model));
    return model

sentence = st.text_input("Enter english text:")
if(st.button("Generate code")): 
    if sentence is not None:
          model=load_model(BASE_DIR)
          model.eval()
          output_words,_ = helper.translate_sentence(sentence,src_stoi,trg_stoi,trg_itos, model, device)
          # Format and print response sentence
          output_words[:] = [x for x in output_words if not (x == '<eos>' or x == '<pad>')]
          st.code('Python Code Generated:\n' + ' '.join(output_words))

Overwriting /content/drive/MyDrive/seq2py/score_glove_augmentation.py


In [ ]:
!nohup streamlit run /content/drive/MyDrive/seq2py/score_glove_augmentation.py &

nohup: appending output to 'nohup.out'


In [ ]:
!cat /content/drive/MyDrive/seq2py/nohup.out


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.202.212.58:8501



https://stackoverflow.com/questions/64918649/streamlit-with-colab-and-pyngrok-failed-to-complete-tunnel-connection-versio

In [ ]:
#Reference

# https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc#:~:text=In%20case%20you%20want%20to,and%20then%20kill%20that%20process.
# !ps aux | grep streamlit
# !kill
# ngrok.get_tunnels()
# ngrok.disconnect('https://7eb62c71f4e9.ngrok.io')
#ngrok.kill()

In [ ]:
ngrok.disconnect('https://95555438847d.ngrok.io/')

In [ ]:
!ps aux | grep streamlit | head -1 | awk -F ' ' '{print $2}' | xargs kill 

In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
!kill 2470364

/bin/bash: line 0: kill: (2470364) - No such process


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("1pVsWrMYOHo5XZccFlZ8N6qejUh_4Jngiuw6qSk89v94wPyCG")
url = ngrok.connect(port=8501)
url

'http://db166dad397d.ngrok.io'

In [ ]:
!cat nohup.out


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://104.199.151.114:8501

